# MBTI Prediction

In [24]:
import pandas as pd
import numpy as np

df = pd.read_csv("./data/mbti_1.csv", header = 0)

df.head()


,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


## Dataset Analysis

## Learning

In [10]:
data = df
print(data.shape)
posts = data['posts']
types = data['type']
# print(posts.iloc[0])

(8675, 2)


### Split dataset in train and test sets

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(posts, types, test_size=0.4, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(5205,) (5205,)
(3470,) (3470,)


### Classification

#### Naive Bayesian

In [22]:
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
                    ])

text_clf = text_clf.fit(X_train, y_train)

### Prediction

In [25]:
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.20720461095100864

#### SVM

In [27]:
from sklearn.linear_model import SGDClassifier

text_clf_svm = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(max_iter=1000, tol=1e-3)),
                    ])

text_clf_svm = text_clf_svm.fit(X_train, y_train)

predicted_svm = text_clf_svm.predict(X_test)
np.mean(predicted_svm == y_test)

0.6619596541786743